# BEE 4750 Homework 2: Systems Modeling and Simulation

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 09/19/24, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to derive a model for water quality in a river
    system and use this model to check for regulatory compliance.
-   Problem 2 asks you to explore the dynamics and equilibrium stability
    of the shallow lake model under a particular set of parameter
    values.
-   Problem 3 (5750 only) asks you to modify the lake eutrophication
    example from Lecture 04 to account for atmospheric deposition.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
Pkg.add("DifferentialEquations")

: 

In [ ]:
using Plots
using LaTeXStrings
using CSV
using DataFrames
using Roots

: 

## Problems (Total: 50/60 Points)

### Problem 1 (25 points)

A river which flows at 10 km/d is receiving discharges of wastewater
contaminated with CRUD from two sources which are 15 km apart, as shown
in the Figure below. CRUD decays exponentially in the river at a rate of
0.36 $\mathrm{d}^{-1}$.

<figure>
<img src="attachment:figures/river_diagram.png"
alt="Schematic of the river system in Problem 1" />
<figcaption aria-hidden="true">Schematic of the river system in Problem
1</figcaption>
</figure>


**In this problem**:

-   Assuming steady-state conditions, derive a model for the
    concentration of CRUD downriver by solving the appropriate
    differential equation(s) analytically.
-   Determine if the system in compliance with a regulatory limit of 2.5
    kg/(1000 m$^3$).

#### Problem 1 Model Setup
##### Model Assumptions
- Assume steady state conditions.
- Assume differential equation(s) are to be implemented in hw02.
- The source begins on the left side at an assumed starting distance of 0 [km], and assigned two variables to the two CRUD contaminated wastewater discharge sources.
    - Leftmost CRUD wastewater source:
        - Discharge rate, source_1 = 40000 [m^(3)/day] 
        - Concentration, conc_1 = 9 [kg/(1000 m^(3))]
    - Rightmost CRUD wastewater source:
        - Discharge rate, source_2 = 60000 [m^(3)/day]
        - Concentration, conc_2 = 7 [kg/(1000 m^(3))]
- Problem statement clearly says,"two sources which are 15 km apart," but does not define a variable for the river inflow to the 1st discharge source. Therefore, it is reasonable to assume that the simulation will run over a numerically unspecified distance of 0 to x + 15. This effect of accounting for variable simulation distances is specified analytically and symbolically in the following sections. 

##### Decision Variables (Units and Meaning)
- River inflow:
     - Flow rate, flow_rate_r = 250000 [m^(3)/day] 
     - River inlet concentration, conc_r = 0.5 [kg/(1000 m^(3))]
- River flow velocity, flow_velocity_r = 10 [km/day]
 - CRUD exponential decay rate, decay_crud = 0.36
 - Distance between wastewater sources, distance_sources = 15 [km]
- Total distance, distance_total range = (0, x + 15) [km]
- Leftmost CRUD wastewater source:
    - Discharge rate, source_1 = 40000 [m^(3)/day] 
    - Concentration, conc_1 = 9 [kg/(1000 m^(3))]
- Rightmost CRUD wastewater source:
    - Discharge rate, source_2 = 60000 m^(3)/day, 
    - Concentration, conc_2 = 7 [kg/(1000 m^(3))]

##### Derivation for Objective 
- Objective: derive a model for the concentration of CRUD downriver from the river inflow from 2 sources, source_1 and source_2 (as defined above) by implementing differential equations to account for changes as the water flows in terms of the distance downriver.
    - Component 1: calculate the concentration of CRUD in the river for 2 segments based on both wastewater source inlets.
        - For segment 1, the distance range is (0, x) [km].
        - For segment 2, the distance range is (x, x + 15) [km]
    - Component 2: implement the CRUD discharge segments into the mass balance at the specified distances.
    - Component 3: run the code from the river inflow at a distance of 0 [km] to the end distance of x + 15 [km]to calculate all relevant output decision variables. 
    - Component 4: running the code will also solve for the concentration of CRUD downriver and then determine whether compliance with a regulatory CRUD concentration limit of [2.5kg/(1000 m)] has been met.

##### Mathematical Derivation for Objective
- Total simulation distance:   
    - distance_total [km] = final distance [km] - river inflow distance [km]
        - distance_total [km] = x + 15 [km] - 0 [km]
        - distance_total [km] = x + 15 [km]
    - Interpretation of total distance derivation: the minimum distance of x + 15 [km] accounts for the distance of 15 [km] between the 2 wastewater discharge sources.

- Mass balance (segment 1):
    - dC/dx = ((inflow concentration [kg/day] + total segment discharge [kg/day])) / (total segment flow rate [m^(3)/day]) - CRUD decay rate * source 1 concentration [kg/(1000 m^(3))]
        - Where inflow concentration [kg/day] = river flow rate [m^(3)/day] * river concentration [kg/(1000 m^(3))],
        - total segment discharge [kg/day] = source 1 CRUD discharge rate [m^(3)/day] * source 1 CRUD discharge concentration [kg/(1000 m^(3))],
        - and total segment flow rate [m^(3)/day] = river flow rate [m^(3)/day] + source 1 CRUD discharge rate [m^(3)/day]
        - Interpretation of mass balance (segment 1): the derivative of concentration C, with respect to x, shows the variable concentration of CRUD discharge downriver from the river inflow and accounts for the source 1 CRUD discharge inlet flow. The source 1 CRUD discharge is multiplied by the CRUD decay rate to represent a loss in CRUD discharge concentration, which is then subtracted from the total discharge in the segment to solve for the total change in concentration over a change in distance, starting at 0 [km] and ending at the CRUD discharge source 1 distance of x [km].

- Mass balance (segment 2):
    - dC/dx = ((inflow concentration [kg/day] + total simulation discharge [kg/day])) / (total simulation flow rate [m^(3)/day]) - CRUD decay rate * simulation concentration [kg/(1000 m^(3))]
        - Where total simulation discharge [kg/day] = (source 1 CRUD discharge rate [m^(3)/day] * source 1 CRUD discharge concentration [kg/(1000 m^(3))]) + (source 2 CRUD discharge rate [m^(3)/day] * source 2 CRUD discharge concentration [kg/(1000 m^(3))]), 
        - and total flow rate [m^(3)/day] = river flow rate [m^(3)/day] + source 1 CRUD discharge rate [m^(3)/day] + source 2 CRUD discharge rate [m^(3)/day]
        - Interpretation of mass balance (segment 2): the derivative of concentration C, with respect to x in segment 2, still shows the variable concentration of CRUD discharge downriver from the river inflow, but now account accounts for both the source 1 and source 2 CRUD discharge inlet flow due to the accumulating nature of this simulation (i.e. the river must add onto its previous concentration from segment 1 rather than re-initializing).

##### Constraints 
- Compliance with regulatory limit = 2.5 [kg/(1000 m)]
    - Interpretation: 
        - If system output is less than or equal to 2.5 [kg/(1000 m)],
            - Then yes, the system is in compliance with a regulatory limit of 2.5 [kg/(1000 m)].
        - If system output is more than 2.5 [kg/(1000 m)],
            - Then no, the system is not in compliance with a regulatory limit of 2.5 [kg/(1000 m)].
- The model is formulated in terms of distance downriver, rather than in terms of time from discharge.

##### Final Answer
- The final answer shows that the CRUD concentration is variable over the total distance of the river, x + 15 [km]. The output based on solving the differential equation prints the term,"CRUD Wastewater Discharge Regulatory Limit Exceeded". 

##### Specific Reference to Modeling Results
- Chose a step size of 0.005 when solving the differential equation and saving. Saving at each step size of 0.05 provided adequate, but still thorough, insights into how the concentration changes over the change in distance.
- Based on the output of the model, the associated CRUD concentration values as distance along the river increased from 0 to x + 15 [km], it is evident that the CRUD concentration changed based on which segment was being solved for at a specific distance as the inflow traveled the full distance of the river. 
    - Segment 1, from 0 to 15 [km]: the river inflow accumulates CRUD discharge concentration from source 1. The model also accounts for the CRUD decay rate in this segment. 
    - Segment 2, from 15 to x + 15 [km]: in this segment, the river inflow accumulates CRUD discharge concentration from source 2 in addition to source 1. Therefore, the total discharge in this segment accounts for the CRUD source discharge from both sources, as well as the overall CRUD decay rate for the sources. The concentration reflects this second inflow by showing a more significant increase in CRUD concentration in segment 2 as compared to segment 1. 
- The printed output,"CRUD Wastewater Discharge Regulatory Limit Exceeded," shows that the limit of 2.5 [kg/(1000 m)] was exceeded by the simulated river model. As such, the regulatory limit for CRUD wastewater discharge was exceeded and therefore does not adhere to the environmental safety regulation. As such, it would be wise to investigate solutions for this issue so that the regulatory limit of 2.5 [kg/(1000 m)] is no longer exceeded. Exceeding the CRUD discharge limit could cause harm to aquatic organisms and habitats, which could result in a reinforcing feedback mechanism that then causes amplifications in system instability. Due to negative aquatic organism and habitat consequences, further river disruption could result in the form of bioaccumulation in those organisms that can be spread to other sources. 
    - Rectification based on Modeling Results: it would be wise to investigate the possibility of reduction in source discharges, as a total of 100,000 [m^(3)/day] flows into the river from both sources. This amount could potentially be diverted and/or reduced to allow for a reduction in CRUD wastewater discharge entering the river. 



In [ ]:
# Load all packages for problem 1
using DifferentialEquations 
using Plots
x = 0

# Initialize variables to be implemented
source_1 = 40000 # [m^(3)/day] 
conc_1 =  9/1000 # [kg/(m^(3))]
source_2 = 60000 # [m^(3)/day]
conc_2 = 7/1000 # [kg/(m^(3))]
flow_rate_r = 250000 # [m^(3)/day]
conc_r = 0.5 # [kg/(1000 m^(3))]
flow_velocity_r = 10 # [km/day]
distance_sources = 15 # [km]
distance_total = x + 15 # [km]
decay_crud = 0.36 # CRUD decay rate, no units applicable
   
# Set up differential equations for calculating CRUD discharge concentration
function crud_river!(du, u, p, x) 

    # Segment 1 value calculations
    if x < 15
        # Inflow concentration, segment 1 discharge, total segment 1 flow rate
        conc_inflow = flow_rate_r * conc_r # [kg/day]
        discharge_s1 = source_1 * conc_1 # [kg/day]
        flow_rate_s1 = flow_rate_r + source_1 # [m^(3)/day]

        # Mass balance to implement in crud_river derivative function
        du[1] = ((conc_inflow + discharge_s1) / (flow_rate_s1)) - (decay_crud * u[1]) # dC/dx
    
    # Segment 2 value calculations
    else
        conc_inflow = flow_rate_r * conc_r # [kg/day]
        discharge_total = (source_1 * conc_1) + (source_2 * conc_2) # [kg/day]
        flow_rate_total = flow_rate_r + source_1 + source_2 # [m^(3)/day]

        # Mass balance to implement in crud_river derivative function
        du[1] = ((conc_inflow + discharge_total) / (flow_rate_total)) - (decay_crud * u[1]) # dC/dx
    end
end

# Initialize variables and define distance 
u0 = [conc_r]
xspan = (0.0, distance_total)

# Setup differential equation, solve, and save at each step size
diffeq_crud = ODEProblem(crud_river!, u0, xspan)
diffeq_crud_output = solve(diffeq_crud, saveat=0.05)

# Retrieve calculated concentration and respective distance value outputs
concs_final = hcat(diffeq_crud_output.u...)'
x_final = diffeq_crud_output.t

# Print calculated concentrations and respective distance value outputs
for i in 1:length(x_final)
    println("Distance = ", x_final[i], "[km]", "CRUD Discharge Concentration =", concs_final[i][1], "[kg/m^(3)]")
end

# Check compliance of simulated values with regulatory CRUD concentration limit
limit_crud = 2.5/1000 # [kg/1000 m^(3)]

# Initialize exceeds limit to be false, check with each iteration
exceeds_limit_crud = false 

# Check concentrations at distance x to see if it exceeds regulatory limit
for i in 1:length(concs_final)
    if concs_final[i][1] > limit_crud
        exceeds_limit_crud = true
        break
    end
end

# Print returned boolean T/F value for exceeding CRUD regulatory limit
if exceeds_limit_crud
    println("CRUD Wastewater Discharge Regulatory Limit Exceeded")
else
    println("CRUD Wastewater Discharge Regulatory Limit Not Exceeded")
end

: 

> **Tip**
>
> Formulate your model in terms of distance downriver, rather than
> leaving it in terms of time from discharge.

### Problem 2 (25 points)

Consider the shallow lake model from class:

$$
\begin{aligned}
X_{t+1} &= X_t + a_t + y_t + \frac{X_t^q}{1 + X_t^q} - bX_t, \\
y_t &\sim \text{LogNormal}(\mu, \sigma^2),
\end{aligned}
$$

where:

-   $X_t$ is the lake phosphorous (P) concentration at time $t$;
-   $a_t$ is the point-source P release at time $t$;
-   $y_t$ is the non-point-source P release at time $t$, which is
    treated as random from a LogNormal distribution with mean $\mu$ and
    standard deviation $\sigma$;
-   $b$ is the linear rate of P outflow;
-   $q$ is a parameter influencing the rate of P recycling from the
    sediment.

**In this problem**:

-   Make an initial conditions plot for the model dynamics for $b=0.5$,
    $q=1.5$, $y_t=0$, and $a_t=0$ for $t=0, \ldots, 30$. What are the
    equilibria? What can you say about the resilience of the system?

    > **Finding equilibria**
    >
    > Use [`Roots.jl`](https://juliamath.github.io/Roots.jl/stable/) to
    > find the equilibria by solving for values where $X_{t+1} = X_t$.
    > For example, if you have functions `X_outflow(X,b)` and
    > `X_recycling(X,q)`, you could create a function
    > `X_delta(x, a) = a + X_recycling(x) - X_outflow(x)` and call
    > `Roots.find_zero(x -> X_delta(x, a), x₀)`, where `x₀` is an
    > initial value for the search (you might need to use your plot to
    > find values for `x₀` near each of the “true” equilibria).

-   Repeat the analysis with $a_t=0.02$ for all $t$. What are the new
    equilibria? How have the dynamics and resilience of the system
    changed?

#### Problem 2 Model Setup
##### Model Assumptions
- Assume that based on the shallow lake model, Phosphorus (P) can vary in concentration with time.
    - These changes may occur because of: point-source P release, non-point-source P release, P recycling from sediment, P outflow.
- Assume steady-state, in which P can vary over time due to point and non-point sources.
- Assume that the variable, y_t, for non-point-source P runoff is random as shown by the y_t [dimensionless] sampling from a LogNormal distribution which shows variations in non-point source P inflow.
- Assume that the variable, X_t [dimensionless], for point-source P release is controllable. 
- Assume that when P concentration is constant over a time t, that is also the time t at which equilibria occurs in the system.
- From lecture, assume the following about P dynamics: 
    - The lake loses P at a linear rate, bX_t (Assumed first-order (linear) decay for lake P).
    - Nutrient cycling reintroduces P from sediment in the form of ((X_t [dimensionless])^(q))/(1+(X_t [dimensionless])^(q)), which is a nonlinear function.
    - From this, it follows that the P level (state) X_(t+1) [dimensionless] is given by the equations X_(t+1) [dimensionless] = X_t [dimensionless] + a_t [dimensionless] + y_t [dimensionless] + ((X_t [dimensionless])^(q))/(1+(X_t [dimensionless])^(q)) - bX_t [dimensionless], where y_t [dimensionless] ~ LogNormal(mu, o^(2))
    - Additionally, lake dynamics without inflows allows the following to hold as true: a_t = y_t = 0, q = 2.5, and b = 0.5

##### Variables (Units and Meaning)
- Parameters:
    - y_t = 0 (non-point-source P release at time t [dimensionless])
    - mu = mean (mean of y_t 's LogNormal distribution)
    - o = standard deviation of y_t 's LogNormal distribution, omikron
    - b = 0.5 = linear rate of P outflow
    - q = 1.5 (parameter influencing rate of P recycling from sediment)
    - t = time range (0,30) [dimensionless t]
- Decision variables:
    - X_t = lake P concentration at time t [dimensionless]
    - a_t = 0 = point-source P release at time t [dimensionless]

##### Derivation for Objective 
- Objective: make an initial conditions plot for the model dynamics for b = 0.5, q = 1.5, y_t = 0, and a_t = 0 [dimensionless] for t (0,30) to analyze the P concentration variations over time in a shallow lake through simulation. Find the equilibria and comment on the resilience of this system. Then repeat the analysis with a_t = 0.02 [dimensionless] for all t, find the new equilibria, and comment on the resilience of the resimulated system that has redefined a_t = 0.02 and how it has or has not changed. 
    - Component 1: model dynamics
        - The model dynamics of this shallow lake system applies the equation X_(t+1) [dimensionless] = X_t [dimensionless] + a_t [dimensionless] + y_t [dimensionless] + ((X_t [dimensionless])^(q))/(1+(X_t [dimensionless])^(q)) - bX_t [dimensionless], which integrates point and non-point sources, sediment, and outflows from the shallow lake system.
    - Component 2: plot the variations in P concentration over time by utilizing Plots.jl. 
        - By applying Plots.jl to the model, it shows how X_t changes based on initialized/defined variables. 
    - Component 4: find equilibria
        - Equilibria involves fixed points of the model dynamics, meaning that the system has no state change. Equilibria occurs where delta(X) = X_(t+1) - X_t = 0, so the outflows and sediment recycling are in balance. However, the equilibria can change as system parameters vary. At steady state, the equilibria equation shows that inflows and recycling are balanced with the outflows of the system and P's concentration is a constant value.  
    - Component 5: repeat the simulation and analysis with a_t = 0.02 [dimensionless] for all t, find the new equilibria, and comment on the resilience of the system and how it has or has not changed. 
        - However, as detailed in component 4, the equilibria can change as system parameters vary.
        - Additionally, resilience is analyzed by investigating how the system equilibria varies over time due to an increased a_t, the point-source P release.

##### Mathematical Derivation for Objective
- Steady State Shallow Lake Model Equation:  
    - X_(t+1) [dimensionless] = X_t [dimensionless] + a_t [dimensionless] + y_t [dimensionless] + ((X_t [dimensionless])^(q))/(1+(X_t [dimensionless])^(q)) - bX_t [dimensionless] = 0
    - Interpretation of derivation: this equation shows that when the shallow lake model system is in steady-state condition, the equilibrium achieved balances the system inputs with the system outputs.
- a_t = 0 and y_t = 0 Equation: 
    - ((X_t [dimensionless])^(q))/(1+(X_t [dimensionless])^(q)) - bX_t [dimensionless] = 0
    - Interpretation of derivation: from this equation, it is appropriate to note that without P inputs of a_t and y_t, the equation is simplified in the aforementioned form. Therefore, sediment recycling and system outflows are still balanced in this equation even without other P inputs. 
- a_t = 0.02   
    - 0.02 + ((X_t [dimensionless])^(q))/(1+(X_t [dimensionless])^(q)) - b * X_t [dimensionless] = 0
    - Interpretation of derivation: this equation is used in the resimulation portion of the shallow lake model. a_t, the point-source P release, is increased from 0 to 0.02, and this equation is applied to the simulation to show how a point-source P release can affect the shallow lake system's P concentrations over time, as well as its' increased values' effects on resilience of the shallow lake. 

##### Constraints 
- Must use Roots.jl to find the equilibria by solving for values where X_(t+1) = X_t
    - Interpretation: Roots.jl is applied in the shallow lake model simulation to find the roots of the system, which is equivalent to the points in time at which the concentration of lake P is unchanging. 

##### Final Answer
- First simulation iteration, where a_t was initialized as a_t = 0:
    - First iteration equilibria 1 = 0.38196601125010543 based on initially defined a_t = 0 value.
    - First iteration equilibria 2 = 1.0000000000000002 based on initially defined a_t = 0 value.
- Second simulation iteration, where a_t was initialized as a_t = 0.02:
    - New equilibria 1 = 0.15442860418310414 based on redefined a_t = 0.02 value.
    - New equilibria 2 = 1.1341008852434815 based on redefined a_t = 0.02 value.

##### Specific Reference to Modeling Results
- Based on the Final Answer section, both initial equilibria values shifted based on the redefining of the a_t variable to 0.02, rather than it's initial value of 0. 
- Implications of bifurcations: system dynamics uncertainties can dramatically change equilibria critical points, which can cause system behavior and equilibrium states to change. Additionally, shocks in the form of sedimentation/recycling disturbances or excessive non-pount source inflows can cause irreversible system outcome alterations. 
    - It is necessary to examine the impacts of bifrucations on shallow lake models because drastic changes in P concentration inflows could cause irreversible damage to the system. As exhibited in the Final Answer section, equilibria 1 was originally equal to 0.38196601125010543 where the P inflow was originally set to 0, but shifted to a value of 0.15442860418310414 where the P inflow was redefined to be set to 0.02. Furthermore, equilibria 2 was originally equal to 1.0000000000000002 where the P inflow was originally set to 0, but shifted to a value of 1.1341008852434815 where the P inflow was redefined to be set to 0.02. This shifting in values represents the value of simulation the shallow lake model, and other similar models, to examine effects of bifurcations such as increasing P inflow concentrations and the potentially irreversible negative effects it could have on the overall system.
- Feedback: reinforcing or dampening of feedbacks cause system state amplifications for instability and evolution away from equilibrium state or reversion to stable equilibrium state.
    - As exhibited in the Final Answer section, equilibria 1 was originally equal to 0.38196601125010543 where the P inflow was originally set to 0, but shifted to a value of 0.15442860418310414 where the P inflow was redefined to be set to 0.02. Furthermore, equilibria 2 was originally equal to 1.0000000000000002 where the P inflow was originally set to 0, but shifted to a value of 1.1341008852434815 where the P inflow was redefined to be set to 0.02. Therefore, a reinforcing feedback mechanism with increased P inflow can drive the shallow lake system further from equilibria and amplify disturbances and overall impacts on its health. Based on the outputs, the shift in this simulation and resimulation was small and thereby exhibits system resiliency since the increasing P inflow a_t did not significantly change the equilibria states of the system. However, further increasing the P inflow concentration, a_t, could reduce system resiliency based on the reinforcing feedback mechanism and thus amplify disturbances to the shallow lake.

In [ ]:
# Load packages
using Plots
using Roots

# Initialize variables 
max_t = 30
b = 0.5
q = 1.5

# Initialize arrays
X = zeros(max_t + 1)
X[1] = 0.1

# Define function 
function shallow_lake_model(X_t, a_t, y_t, b, q)
    return X_t + a_t + y_t + ((X_t)^(q)) / (1 + (X_t^(q))) - b * X_t
end

# 1. Initialize point-source and non-point-source P to 0
y_t = 0 # [dimensionless]
a_t = 0 # [dimensionless]

# Iteration through time until t = 30
for t in 1:max_t
    X[t+1] = shallow_lake_model(X[t], a_t, y_t, b, q)
end

# Plot initial conditions for model dynamics of initial system variables
plot(0:max_t, X, xlabel="Time [t]", ylabel="Phosphorous (P) Lake Concentration", title="Shallow Lake Model Phosphorus Concentration Variations vs. T", legend=false)

# Calculate equilibria of system
function delta_X(X, a, b, q)
    return a + (X^(q)/(1 + X^(q))) - b * X
end

# Find initial equilibria of system
equilibria_1 = find_zero(X -> delta_X(X, 0.0, b, q), 0.2)
equilibria_2 = find_zero(X -> delta_X(X, 0.0, b, q), 3)

println("Equilibria found: a_t = 0, $equilibria_1, $equilibria_1")

# 2. Repeat the analysis with a_t = 0.02
a_t = 0.02 # Redefined [dimensionless] variable definition

# Reinitialize relevant initial conditions
X = zeros(max_t + 1)
X[1] = 0.1

# Rerun the simulation model for redefined a_t variable
for t in 1:max_t
    X[t+1] = shallow_lake_model(X[t], a_t, y_t, b, q)
end

# Plot initial conditions for model dynamics with redefined a_t value
plot(0:max_t, X, xlabel="Time [t]", ylabel="Phosphorous (P) Lake Concentration", title="Shallow Lake Model Phosphorus Concentration Variations over Time", legend=false)

# Calculate equilibria of system corresponding to redefined a_t value
equilibria_1_new_at = find_zero(X -> delta_X(X, 0.02, b, q), 0.2)
equilibria_2_new_at = find_zero(X -> delta_X(X, 0.02, b, q), 3)

println("Equilibria found: a_t = 0, $equilibria_1_new_at, $equilibria_1_new_at")

: 

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

Consider the lake eutrophication example from [Lecture
04](https://viveks.me/environmental-systems-analysis/slides/lecture03-1-eutrophication-modeling.html#/title-slide).
Suppose that phosphorous is also atmospherically deposited onto the lake
surface at a rate of
$1.6 \times 10^{-4} \mathrm{kg/(yr} \cdot \mathrm{m}^2)$, which is then
instantly mixed into the lake. Derive a model for the lake phosphorous
concentration and find the maximum allowable point source phosphorous
loading if the goal is to keep lake concentrations below 0.02 mg/L.

## References

List any external references consulted, including classmates.

- https://docs.julialang.org/en/v1/manual/functions/
    - Used to find information for calling hcat, expression [A B C ...] (problem 1)
    - Information on xspan to define the distance x's range from the beginning to the end of the simulation (problem 1).
- https://docs.sciml.ai/DiffEqDocs/stable/
    - Useful for defining differential equations in Julia (problem 1) to calculate the change in concentration with respect to the change in distance. Parameters included an array to store the derivative (du[]), an array of present values for discharge concentration, and distance x of the river.
- https://docs.sciml.ai/DiffEqDocs/stable/types/ode_types/
    - Helped with syntax of differential equations in Julia (problem 1) to calculate the change in concentration with respect to the change in distance, f!(du,u,p,t). 
    - Also found a way to translate differential equations into Julia so it can be solved outright with ODEProblem.
- 
    - solve command syntax to solve the differential equation analytically, and saveat option to solve it at specific distances x (instead of time, as detailed in this resource).